# Семинар 2 – Линейные модели

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(palette='deep', style='darkgrid', rc={"figure.figsize": (15, 4)})
import scipy.stats as st

import warnings
warnings.simplefilter('ignore')

Сгенерируем исскуственные данные, на основе функции:
$$f(x) = 4x+5$$

In [ ]:
def lin_function(x):
    return 4*x+5

x_true = np.array([-2,2])
y_true = lin_function(x_true)


In [ ]:
plt.plot(x_true, y_true, linewidth=1)
plt.show()

In [ ]:
n = 100
x = np.random.rand(n,1)*4-2
e = np.random.rand(n,1)*4-2
y = lin_function(x) + e


In [ ]:
plt.scatter(x, y, color='g')
plt.plot(x_true, y_true, linewidth=1)
plt.show()

# Аналитический метод
$$\hat \theta = \bigl(X^T \cdot X  \bigr)^{-1} \cdot X^T \cdot y $$

In [ ]:
x_matrix = np.c_[np.ones((n,1)),x]

In [ ]:
%%time
thetha_matrix = np.linalg.inv(x_matrix.T.dot(x_matrix)).dot(x_matrix.T).dot(y)

In [ ]:
thetha_matrix.T[0].tolist()

In [ ]:
print("Свободный член: {[0][0]:.7}".format(thetha_matrix.T))
print("Коэфициент: {[0][1]:.7}".format(thetha_matrix.T))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
%%time
lr = LinearRegression()
lr.fit(x,y)

In [ ]:
print("Свободный член: {:.7}".format(lr.intercept_[0]))
print("Коэфициент: {:.7}".format(lr.coef_[0][0]))

In [ ]:
plt.scatter(x, y, color='g')
plt.scatter(x, lr.predict(x), color='r')
plt.plot(x_true, y_true, linewidth=1)
plt.show()

# Пакетный градиентный спуск

$$\nabla MSE(\theta)= \frac{2}{l} X^T \cdot \bigl(X \cdot \theta - y \bigr) $$

### Реализация в numpy

In [ ]:
%%time
eta = 0.1  # learning rate
n_iterations = 100


theta = np.random.randn(2,1)  # random initialization

plt.scatter(x, y, color='g')

for iteration in range(n_iterations):
    if iteration < 10:
        plt.plot(x_true, x_true*theta[1]+theta[0], linewidth=1, color='r')
    gradients = 2/n * x_matrix.T.dot(x_matrix.dot(theta) - y)
    theta = theta - eta * gradients

plt.plot(x_true, y_true, linewidth=1)
plt.show()

print(theta)

### Слишком маленький шаг обучения (learning rate)

In [ ]:
eta = 0.01  # learning rate
n_iterations = 100


theta = np.random.randn(2,1)  # random initialization

plt.scatter(x, y, color='g')

for iteration in range(n_iterations):
    if iteration < 10:
        plt.plot(x_true, x_true*theta[1]+theta[0], linewidth=1, color='r')
    gradients = 2/n * x_matrix.T.dot(x_matrix.dot(theta) - y)
    theta = theta - eta * gradients

plt.plot(x_true, y_true, linewidth=1)
plt.show()

### Слишком большой шаг обучения (learning rate)

In [ ]:
eta = 1.01  # learning rate
n_iterations = 100


theta = np.random.randn(2,1)  # random initialization

plt.scatter(x, y, color='g')

for iteration in range(n_iterations):
    if iteration < 10:
        plt.plot(x_true, x_true*theta[1]+theta[0], linewidth=1, color='r')
    gradients = 2/n * x_matrix.T.dot(x_matrix.dot(theta) - y)
    theta = theta - eta * gradients

plt.plot(x_true, y_true, linewidth=1)
plt.show()

# Уменьшение шага на каждой итерации

In [ ]:
eta = 1  # learning rate
n_iterations = 1000

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/n * x_matrix.T.dot(x_matrix.dot(theta) - y)
    theta = theta - (eta/(iteration+1)) * gradients


print(theta)

Learning rate - гипперпараметр, и можно воспользоваться GridSearchCV, однако чтобы не учить каждый раз такое кол-во итераций, мы можем измерять норму градиента, и прекращать спуск, когда он "затух"

In [ ]:
eta = 0.1  # learning rate
n_iterations = 1000
tol = 0.00001

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/n * x_matrix.T.dot(x_matrix.dot(theta) - y)
    if np.linalg.norm(gradients) < tol:
        break
    theta = theta - eta * gradients

print('Градиент затух на {} итерации '.format(iteration))
print(theta)

__Реализация в Scikit-Learn отсутствует__
  
  
# Cтохастический градиентный спуск  

In [ ]:
n_epochs = 50

def learning_schedule(t):
    return t0 / (t + t1)

t0, t1 = 5, 100  # learning schedule hyperparameters


theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(n):
        random_index = np.random.randint(n)
        xi = x_matrix[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * n + i)
        theta = theta - eta * gradients
print(theta)

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
sgd = SGDRegressor(tol=0.0001)
#The stopping criterion. If it is not None, the iterations will stop when (loss > previous_loss - tol).
sgd.fit(x,y)
sgd.intercept_, sgd.coef_

# Влияние выбора функции потерь
По мотивам [Семинара курса ML на ФИВТ](https://github.com/ml-mipt/ml-mipt/blob/master/week02_linear_reg/week02_linear_regression.ipynb)

In [ ]:
with open('../data/data_preprocessed.json') as file:
    X = pd.read_json(file)

In [ ]:
X_subset = X[[7, 15]].values
# add two outliers
X_subset_modified = np.vstack((X_subset, [[1, 90], [2, 50]]))

In [ ]:
def scatter_points_and_plot_line_MSE(X_subset):
    plt.scatter(X_subset[:, 0], X_subset[:, 1])
    lr = LinearRegression()
    lr.fit(X_subset[:, 0][:, None], X_subset[:, 1])
    grid = np.linspace(0, 2, 100)
    line = lr.predict(grid[:, None])
    plt.plot(grid, line)

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
scatter_points_and_plot_line_MSE(X_subset)
plt.title('Без шумовых признаков')
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
scatter_points_and_plot_line_MSE(X_subset_modified)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.title('С шумовыми признаками')
plt.show()

Из-за шумовых объектов прямая достаточно сильно изменила наклон. Поэтому вместо MSE часто используют Mean Absoulte Error: $$L(y_i, a(x_i)) = |y_i - a(x_i)|$$
Теперь обучим регрессию, оптимизируя MAE. В sklearn такая регрессия не реализована, но можно использовать модуль statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
def scatter_points_and_plot_line_MAE(X_subset):
    mod = smf.quantreg('f15 ~ f7', pd.DataFrame(data=X_subset, columns=["f7", "f15"])) # задаеем зависимость и передаем данные
    res = mod.fit(q=0.5)
    plt.scatter(X_subset[:, 0], X_subset[:, 1])   # визуализируем точки
    grid = np.linspace(0, 2, 100)
    plt.plot(grid, grid * res.params["f7"] + res.params["Intercept"])   # визуализируем прямую
    return mod, res

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.title('Без шумовых признаков')
model, result = scatter_points_and_plot_line_MAE(X_subset)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
plt.title('С шумовыми признаками')
model, result = scatter_points_and_plot_line_MAE(X_subset_modified)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.show()

Прямая не изменила направление из-за выбросов.
Попробуем добавить больше шумовых объектов:

In [ ]:
X_subset_modified_twice = np.vstack((
    X_subset_modified, 
    np.random.randint(5, size=60).reshape(-1, 2) * [1, 30],
))

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
model, result = scatter_points_and_plot_line_MAE(X_subset)
plt.title('Без шумовых признаков')
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
model, result = scatter_points_and_plot_line_MAE(X_subset_modified_twice)
plt.title('С шумовыми признаками')
plt.ylim(-20, 100)
plt.xlabel("x")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error

Для примера решения задачи прогнозирования, я взял набор данных Energy efficiency из крупнейшего репозитория UCI.   

В нем $X_1 ... X_8$ — характеристики помещения на основании которых будет проводиться анализ, а $y_1,y_2$ — значения нагрузки, которые надо спрогнозировать.
- $X_1$	Относительная компактность
- $X_2$	Площадь
- $X_3$	Площадь стен
- $X_4$	Площадь потолка	
- $X_5$	Общая высота	
- $X_6$	Ориентация
- $X_7$	Площадь остекления	
- $X_8$	Распределенная площадь остекления	
- $y_1$	Нагрузка при обогреве
- $y_2$	Нагрузка при охлаждении

In [ ]:
data = pd.read_csv('../data/energy_efficiency.csv')
data.head()

Рассмотрим Матрицу корреляции

In [ ]:
data.corr()

Увеличим читаемость:

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = data.drop(['Y1','Y2'], axis=1).corr()
sns.heatmap(corr, square=True, ax=ax, cmap=sns.diverging_palette(220, 10, as_cmap=True))
plt.show()

In [ ]:
X = data.drop(['Y1','Y2'], axis=1)
y = data['Y1']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def my_linear_regression(X_train, Y_train):
    return np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)

def predict(X, w):
    return np.dot(X, w)

In [ ]:
w = my_linear_regression(X_train, y_train)
print(w)
y_train_pred = predict(X_train, w)
print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))

Удалим скоррелированные признаки

In [ ]:
X_droped = data.drop(['X1','X4', 'Y1','Y2'], axis=1)
y = data['Y1']
X_train_new, X_test_new, y_train, y_test = train_test_split(X_droped, y, test_size=0.2, random_state=42)

In [ ]:
w = my_linear_regression(X_train_new, y_train)
print(w)
y_train_pred = predict(X_train_new, w)
print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test_new, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))

Применим регуляризацию:

In [ ]:
def my_linear_regression(X_train, Y_train, l2=0):
    return np.linalg.inv(X_train.T.dot(X_train) + l2*np.eye(X_train.shape[1])).dot(X_train.T).dot(y_train)

In [ ]:
w = my_linear_regression(X_train, y_train, 1)
print(w)
y_train_pred = predict(X_train, w)
print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))

# Посмотрим, как внутри sklearn

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
print(linear_model.coef_)
y_train_pred = linear_model.predict(X_train)
y_test_pred = linear_model.predict(X_test)

print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))

In [ ]:
linear_model = Ridge()
linear_model.fit(X_train, y_train)
print(linear_model.coef_)
y_train_pred = linear_model.predict(X_train)
y_test_pred = linear_model.predict(X_test)

print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))

In [ ]:
linear_model = Lasso()
linear_model.fit(X_train, y_train)
print(linear_model.coef_)
y_train_pred = linear_model.predict(X_train)
y_test_pred = linear_model.predict(X_test)

print('='*10, 'Train', '='*10)
print("MSE: ", mean_squared_error(y_train, y_train_pred))
print("R2: ", r2_score(y_train, y_train_pred))
y_test_pred = predict(X_test, w)
print('='*10, 'Test', '='*10)
print("MSE: ", mean_squared_error(y_test, y_test_pred))
print("R2: ", r2_score(y_test, y_test_pred))